# Convert all sports dfs into one

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, time
today = pd.Timestamp.today().date()

import pytz

## Read

In [2]:
# Get the current year
current_year = datetime.now().year
next_year = current_year + 1

# Define a custom date parser for the corresponding formats
fb_date_parser = lambda x: pd.to_datetime(x, format="%d/%m/%y").date()
nba_nhl_date_parser = lambda x: pd.to_datetime(x, format="%a, %b %d, %Y").date()
nfl_date_parser = lambda x: pd.to_datetime(f"{x}, {next_year}" if "Jan" in x else f"{x}, {current_year}", format="%a, %b %d, %Y").date() if pd.notna(x) else np.nan


# Function for home/away cols for nba/nhl/nfl
def assign_teams(df, home_away_col, user_team_col, opponent_col):
    df['Home Team'] = np.where(df[home_away_col] == 'vs.', df[user_team_col], df[opponent_col])
    df['Away Team'] = np.where(df[home_away_col] == '@', df[user_team_col], df[opponent_col])
    return df

In [3]:
# Read in files with their dates
## football/soccer
df_fb = pd.read_csv(
    r"C:\Users\Owner\Documents\Data Projects\GitHub\Apps\project_w\2024_10_28__all_in_one\data\df_fb_master_2024_10_28.csv",
    parse_dates=['Date'], 
    date_parser=fb_date_parser
)
### Create/convert cols as necessary
df_fb['Date'] = pd.to_datetime(df_fb['Date']).dt.date
df_fb = df_fb[df_fb['Date'] >= today]
df_fb['Time'] = df_fb['Time'].astype(str).str.replace(r'\s+', '', regex=True)
df_fb['Time'] = df_fb['Time'].replace('-', pd.NA)
df_fb['Time'] = pd.to_datetime(df_fb['Time'], format='%H:%M').dt.time


## nba
df_nba = pd.read_csv(
    r"C:\Users\Owner\Documents\Data Projects\GitHub\Apps\project_w\2024_10_28__all_in_one\data\df_nba_master_2024_10_29.csv",
    parse_dates=['Date'], 
    date_parser=nba_nhl_date_parser
)
### Create/convert cols as necessary
df_nba['Date'] = pd.to_datetime(df_nba['Date']).dt.date
df_nba = assign_teams(df_nba, 'Home/Away', 'user_team', 'Opponent')
df_nba['Start (ET)'] = pd.to_datetime(df_nba['Start (ET)'], infer_datetime_format=True).dt.time
df_nba = df_nba.rename(columns={'Start (ET)': 'Time'})
df_nba['Time'] = pd.to_datetime(df_nba['Time'].astype(str)) - pd.to_timedelta(2, unit='hours')
df_nba['Time'] = df_nba['Time'].dt.time


## nhl
df_nhl = pd.read_csv(
    r"C:\Users\Owner\Documents\Data Projects\GitHub\Apps\project_w\2024_10_28__all_in_one\data\df_nhl_master_2024_10_30.csv",
    parse_dates=['Date'], 
    date_parser=nba_nhl_date_parser
)
### Create/convert cols as necessary
df_nhl['Date'] = pd.to_datetime(df_nhl['Date']).dt.date
df_nhl = assign_teams(df_nhl, 'Home/Away', 'user_team', 'Opponent')
df_nhl['Time'] = pd.to_datetime(df_nhl['Time'], infer_datetime_format=True).dt.time
df_nhl['Time'] = pd.to_datetime(df_nhl['Time'].astype(str)) - pd.to_timedelta(2, unit='hours')
df_nhl['Time'] = df_nhl['Time'].dt.time


## nfl
df_nfl = pd.read_csv(
    r"C:\Users\Owner\Documents\Data Projects\GitHub\Apps\project_w\2024_10_28__all_in_one\data\df_nfl_master_2024_11_01.csv",
    parse_dates=['DATE'], 
    date_parser=nfl_date_parser
)
df_nfl['DATE'] = pd.to_datetime(df_nfl['DATE']).dt.date
df_nfl = assign_teams(df_nfl, 'Home/Away', 'user_team', 'OPPONENT')
df_nfl['TIME'] = pd.to_datetime(df_nfl['TIME'], infer_datetime_format=True).dt.time
df_nfl['TIME'] = pd.to_datetime(df_nfl['TIME'].astype(str)) - pd.to_timedelta(2, unit='hours')
df_nfl['TIME'] = df_nfl['TIME'].dt.time

## Combine fb, nba, nhl, nfl

In [4]:
# First, rename and reorder columns in df_fb
df_fb_fin = df_fb.rename(columns={'Team': 'user_team', 'League': 'game_type'})
df_fb_fin = df_fb_fin[['user_team', 'Date', 'game_type', 'Home Team', 'Away Team', 'Time']].copy()
df_fb_fin['Sport'] = 'Soccer'

# Select and reorder columns in df_nba and df_nhl
fin_cols_list = ['user_team', 'Date', 'game_type', 'Home Team', 'Away Team', 'Time']
df_nba_fin = df_nba[fin_cols_list].copy()
df_nba_fin['Sport'] = 'NBA'

df_nhl_fin = df_nhl[fin_cols_list].copy()
df_nhl_fin['Sport'] = 'NHL'

# Select and reorder columns in df_nfl
df_nfl_fin = df_nfl[['user_team','DATE','game_type','Home Team','Away Team', 'TIME']].copy()
df_nfl_fin.columns = fin_cols_list
df_nfl_fin['Sport'] = 'NFL'

In [5]:
# Stack all dataframes and Sort the combined dataframe by Date and then by Time, both in ascending order
df_fin_all = (
    pd.concat([df_fb_fin, df_nba_fin, df_nhl_fin, df_nfl_fin], ignore_index=True)
    .sort_values(by=['Date', 'Time', 'Home Team'], ascending=[True, True, True])
    .reset_index(drop=True)
)

In [6]:
df_fin_future = df_fin_all[df_fin_all['Date'] >= today]
df_fin_future

,user_team,Date,game_type,Home Team,Away Team,Time,Sport
650,Aston Villa,2024-11-06,UCL,Club Brugge,Aston Villa,10:45:00,Soccer
651,Leeds United,2024-11-06,CHA,Millwall,Leeds United,12:45:00,Soccer
652,Bayern Munich,2024-11-06,UCL,Bayern Munich,Benfica,13:00:00,Soccer
653,Arsenal,2024-11-06,UCL,Inter Milan,Arsenal,13:00:00,Soccer
654,Inter Milan,2024-11-06,UCL,Inter Milan,Arsenal,13:00:00,Soccer
...,...,...,...,...,...,...,...
7535,Toulouse,2025-05-17,LI1,Saint-Etienne,Toulouse,NaT,Soccer
7536,Strasbourg,2025-05-17,LI1,Strasbourg,Le Havre,NaT,Soccer
7537,FC Cologne,2025-05-18,2.B,FC Cologne,Kaiserslautern,07:30:00,Soccer
7538,Hertha Berlin,2025-05-18,2.B,Hertha Berlin,Hannover 96,07:30:00,Soccer


In [7]:
df_fin_future[df_fin_future['Sport'].isin(['NBA','NHL'])]

,user_team,Date,game_type,Home Team,Away Team,Time,Sport
661,Charlotte Hornets,2024-11-06,regular season,Charlotte Hornets,Detroit Pistons,17:00:00,NBA
662,Detroit Pistons,2024-11-06,regular season,Charlotte Hornets,Detroit Pistons,17:00:00,NBA
663,Indiana Pacers,2024-11-06,regular season,Indiana Pacers,Orlando Magic,17:00:00,NBA
664,Orlando Magic,2024-11-06,regular season,Indiana Pacers,Orlando Magic,17:00:00,NBA
665,Nashville Predators,2024-11-06,regular season,Washington Capitals,Nashville Predators,17:00:00,NHL
...,...,...,...,...,...,...,...
7351,Ottawa Senators,2025-04-17,regular season,Ottawa Senators,Carolina Hurricanes,17:00:00,NHL
7352,Pittsburgh Penguins,2025-04-17,regular season,Pittsburgh Penguins,Washington Capitals,17:00:00,NHL
7353,Washington Capitals,2025-04-17,regular season,Pittsburgh Penguins,Washington Capitals,17:00:00,NHL
7354,Detroit Red Wings,2025-04-17,regular season,Toronto Maple Leafs,Detroit Red Wings,17:00:00,NHL


In [8]:
df_fin_future[df_fin_future['Sport'] == 'NFL']

,user_team,Date,game_type,Home Team,Away Team,Time,Sport
730,Cincinnati Bengals,2024-11-07,regular season,Baltimore,Cincinnati Bengals,18:15:00,NFL
731,Baltimore Ravens,2024-11-07,regular season,Baltimore Ravens,Cincinnati,18:15:00,NFL
877,Carolina Panthers,2024-11-10,regular season,Carolina Panthers,New York *,07:30:00,NFL
878,New York Giants,2024-11-10,regular season,New York Giants,Carolina *,07:30:00,NFL
896,New England Patriots,2024-11-10,regular season,Chicago,New England Patriots,11:00:00,NFL
...,...,...,...,...,...,...,...
3037,Philadelphia Eagles,2024-12-29,regular season,Philadelphia Eagles,Dallas,14:25:00,NFL
3058,Miami Dolphins,2024-12-29,regular season,Cleveland,Miami Dolphins,18:20:00,NFL
3059,Cleveland Browns,2024-12-29,regular season,Cleveland Browns,Miami,18:20:00,NFL
3084,Detroit Lions,2024-12-30,regular season,San Francisco,Detroit Lions,18:15:00,NFL


In [9]:
df_fin_future

,user_team,Date,game_type,Home Team,Away Team,Time,Sport
650,Aston Villa,2024-11-06,UCL,Club Brugge,Aston Villa,10:45:00,Soccer
651,Leeds United,2024-11-06,CHA,Millwall,Leeds United,12:45:00,Soccer
652,Bayern Munich,2024-11-06,UCL,Bayern Munich,Benfica,13:00:00,Soccer
653,Arsenal,2024-11-06,UCL,Inter Milan,Arsenal,13:00:00,Soccer
654,Inter Milan,2024-11-06,UCL,Inter Milan,Arsenal,13:00:00,Soccer
...,...,...,...,...,...,...,...
7535,Toulouse,2025-05-17,LI1,Saint-Etienne,Toulouse,NaT,Soccer
7536,Strasbourg,2025-05-17,LI1,Strasbourg,Le Havre,NaT,Soccer
7537,FC Cologne,2025-05-18,2.B,FC Cologne,Kaiserslautern,07:30:00,Soccer
7538,Hertha Berlin,2025-05-18,2.B,Hertha Berlin,Hannover 96,07:30:00,Soccer


## Save the data

In [20]:
import os
from datetime import datetime

# Get today's date
today_date = datetime.now().strftime("%Y_%m_%d")

# Define the file name
file_name = f"df_fin_future_{today_date}.csv"

# Construct the path using the environment variable
save_path = os.path.join(os.getenv('PROJECT_W_ALL_IN_ONE_DATA'), file_name)

# Save the DataFrame
df_fin_future.to_csv(save_path, index=False)
